# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np

In [ ]:
#| export
class Variable:
    def __init__(self, data):
        if data is not None: # Why allowing "None" here?
            if not isinstance(data, np.ndarray):
                raise TypeError("{} is not supported".format(type(data)))
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad == None:
            self.grad = np.ones_like(self.data)
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
            if x.creator == None:
                continue
            funcs.append(x.creator)

In [ ]:
# Test Variable
x = Variable(np.array(1.0))
assert x.data == np.array(1.0)

x = Variable(None)
assert x.data == None

try:
    Variable(1.0)
except Exception as e:
    assert f"{e}"=="<class 'float'> is not supported"

In [ ]:
#| export
class Function:
    def __call__(self, inputs):
        def as_array(y): return np.array(y) if np.isscalar(y) else y # for numpy spec

        xs = [input.data for input in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)

        self.inputs = inputs
        self.outputs = outputs
        return outputs

    def forward(self, in_data):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

In [ ]:
#| export
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,)

def add(xs):
    return Add()(xs)

In [ ]:
xs = [Variable(np.array(2)), Variable(np.array(3))]
ys = add(xs)
y = ys[0]
assert y.data==5

In [ ]:
#| export
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

def square(x):
    return Square()(x)

In [ ]:
# Test Square
x = Variable(np.array(10))
f = Square()
y = f(x)
assert type(y)==Variable
assert y.data==100

In [ ]:
#| export
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

def exp(x):
    return Exp()(x)

In [ ]:
# Test Exp
x = Variable(np.array(2.))
y = exp(x)
assert np.allclose(y.data, 7.3890561)

In [ ]:
# Test concatenate
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)
assert np.allclose(y.data, 1.648721270700128)

In [ ]:
#| export
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(np.array(x.data - eps))
    x1 = Variable(np.array(x.data + eps))
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

In [ ]:
# test numerical_diff
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)

assert np.allclose(dy, 4)

In [ ]:
# concatenate test with numerical_diff
def f(x): return square(exp(square(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
assert np.allclose(dy, 3.2974426293330694)

In [ ]:
# Test backward
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

y.backward()
assert np.allclose(x.grad, 3.297442541400256)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()